# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where T
    lattice=T(a) * I(3)  # lattice is a cube of ``a`` Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...), tol=tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770866668686                   -0.52    9.0
  2   -2.772148218628       -2.89       -1.32    1.0
  3   -2.772169838118       -4.67       -2.36    2.0
  4   -2.772170722480       -6.05       -4.02    2.0
┌ Warning: Negative ρ detected
│   min_ρ = -5.2044533447966974e-18
└ @ DFTK /home/runner/work/DFTK.jl/DFTK.jl/src/densities.jl:7
  5   -2.772170723003       -9.28       -4.98    2.0
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770797326849                   -0.52    9.0
  2   -2.772059399389       -2.90       -1.32    1.0
  3   -2.772083167145       -4.62       -2.50    1.0
  4   -2.772083364103       -6.71       -3.23    2.0
  5   -2.772083413384       -7.31       -3.69    2.0
  6   -2.772083417692       -8.37       -4.59    2.0


1.77364289369745

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770830543964                   -0.52    9.0
  2   -2.772063002159       -2.91       -1.32    1.0
  3   -2.772082618494       -4.71       -2.37    2.0
  4   -2.772083417599       -6.10       -3.95    2.0
  5   -2.772083417795       -9.71       -4.75    1.0

Polarizability via ForwardDiff:       1.77253801587691
Polarizability via finite difference: 1.77364289369745
